In [9]:
import random

def XOR(bits1: str, bits2: str) -> str:
    """Bitwise XOR of two equal-length bit strings."""
    return ''.join('1' if b1 != b2 else '0' for b1, b2 in zip(bits1, bits2))

def f_function(block64: str, subkey64: str) -> str:
    """
    Example round function that:
      1) Rotates the 64-bit block left by 3 bits.
      2) XORs with the 64-bit subkey.
      3) Reverses the result as a simple permutation.
    """
    rotated = block64[3:] + block64[:3]     # Simple rotation
    xored   = XOR(rotated, subkey64)       # XOR with subkey
    return xored[::-1]                     # Reverse as a toy permutation

def feistel_round(left: str, right: str, subkey: str) -> (str, str):
    """
    One round of the Feistel network:
      - F function on 'right' with 'subkey'.
      - XOR F output with 'left'.
      - Swap roles for the next round.
    """
    fout = f_function(right, subkey)
    new_left  = right
    new_right = XOR(left, fout)
    return new_left, new_right

def generate_subkeys(key192: str, num_rounds: int = 16) -> list[str]:
    """
    Generate a list of 64-bit round subkeys from the 192-bit master key.
    Very simple approach:
      - Each round picks a different 64-bit window from the key (with wrapping).
    """
    if len(key192) != 192:
        raise ValueError("Master key must be exactly 192 bits (string of '0'/'1' of length 192).")
    subkeys = []
    for i in range(num_rounds):
        # For round i, shift by 32*i bits (wrapping) and take 64 bits
        offset  = (32 * i) % 192
        shifted = key192[offset:] + key192[:offset]
        subkey  = shifted[:64]
        subkeys.append(subkey)
    return subkeys

def feistel_encrypt(plaintext128: str, key192: str, num_rounds: int = 16) -> str:
    """
    Encrypt a 128-bit block using a 192-bit master key via a Feistel network.
    """
    if len(plaintext128) != 128:
        raise ValueError("Plaintext must be exactly 128 bits.")
    left, right = plaintext128[:64], plaintext128[64:]
    subkeys = generate_subkeys(key192, num_rounds)
    for i in range(num_rounds):
        left, right = feistel_round(left, right, subkeys[i])
    # Final swap in a Feistel-based cipher is typically optional:
    return right + left

def feistel_decrypt(ciphertext128: str, key192: str, num_rounds: int = 16) -> str:
    """
    Decrypt a 128-bit block using a 192-bit master key via a Feistel network.
    """
    if len(ciphertext128) != 128:
        raise ValueError("Ciphertext must be exactly 128 bits.")
    left, right = ciphertext128[:64], ciphertext128[64:]
    subkeys = generate_subkeys(key192, num_rounds)
    # Decryption uses the same round function but in reverse key order
    for i in reversed(range(num_rounds)):
        left, right = feistel_round(left, right, subkeys[i])
    return right + left

def demo_encryption_decryption():
    print("=== Demo: Encryption & Decryption ===")
    # Generate random 128-bit plaintext
    plaintext128 = ''.join(random.choice('01') for _ in range(128))
    # Generate random 192-bit key
    key192       = ''.join(random.choice('01') for _ in range(192))
    
    ciphertext   = feistel_encrypt(plaintext128, key192)
    decrypted    = feistel_decrypt(ciphertext, key192)
    
    print(f"Plaintext:  {plaintext128}")
    print(f"Ciphertext: {ciphertext}")
    print(f"Decrypted:  {decrypted}")
    print("Success!" if plaintext128 == decrypted else "Mismatch!")
    print()

def demo_IND_CPA():
    """
    A *very simple* demonstration of the IND-CPA concept (not a formal proof):
      1) Define two distinct 128-bit blocks p0 and p1.
      2) Encrypt one of them at random with the same key.
      3) Show the ciphertext to an 'attacker' who must guess which plaintext was used.
    """
    print("=== Demo: IND-CPA Experiment (Toy Example) ===")
    # Two distinct plaintexts
    p0 = "0" * 128
    p1 = "1" * 128
    # Generate random 192-bit key
    key192 = ''.join(random.choice('01') for _ in range(192))
    
    # Encrypt both
    c0 = feistel_encrypt(p0, key192)
    c1 = feistel_encrypt(p1, key192)
    
    # Randomly pick one to show to an 'attacker'
    chosen = random.choice([0, 1])
    challenge_cipher = c0 if chosen == 0 else c1
    print(f"Challenge ciphertext: {challenge_cipher}")
    
    # In a real IND-CPA game, an attacker tries to guess if it was c0 or c1.
    # If the cipher is secure, the best strategy is random guessing => 50% success.
    # We'll just skip the attacker side and show we can't trivially distinguish them.
    print("Attacker's best guess is random. Probability of success = 50%.\n")

# Run both demonstrations
if __name__ == "__main__":
    demo_encryption_decryption()
    demo_IND_CPA()


=== Demo: Encryption & Decryption ===
Plaintext:  11001110111111111111111000011100000010101011000011001100100111110111110110110101110101100000111101101010111110010111100000100001
Ciphertext: 01010001011100000001100001000101101001101101001001001001100000110101011100000100101010001000110011001100101011100110110100011110
Decrypted:  11001110111111111111111000011100000010101011000011001100100111110111110110110101110101100000111101101010111110010111100000100001
Success!

=== Demo: IND-CPA Experiment (Toy Example) ===
Challenge ciphertext: 01011001111010000111010010000101000111100001100110001110000001111101010110001001011010101101111010000101100110111111101100110100
Attacker's best guess is random. Probability of success = 50%.

